Disclaimer: este código não foi feito com a ajuda do ChatGPT, mas sim utilizando a documentação/API Reference da própria Pinecone, OpenAI e Langchain, bem como os exemplos disponibilizados por elas (o que me fez levar mais tempo kk damn these robots!).

In [1]:
import json
import pandas as pd

In [2]:
with open("../articles_scraping/noh_faq_completo_vf.json", "r", encoding="utf-8") as f:
    dados = json.load(f)

In [3]:
df = pd.DataFrame(dados)
df.head(2)

,url,titulo,conteudo,artigos_relacionados,categoria,subtitulo_categoria,link_categoria
0,https://www.noh.com.br/ajuda/o-que-e-a-noh,O que é a Noh?,"A Noh é uma\nconta para o casal que junto, pod...",[https://www.noh.com.br/ajuda/como-funciona-o-...,Sobre Nohs,Saiba mais sobre a Noh e a nossa segurança,https://www.noh.com.br/ajuda-categorias/sobre-...
1,https://www.noh.com.br/ajuda/os-meus-dados-e-o...,Os meus dados e o meu dinheiro estão seguros?,"Sim, os seus dados e dinheiro estão seguros co...",[https://www.noh.com.br/ajuda/como-funciona-o-...,Sobre Nohs,Saiba mais sobre a Noh e a nossa segurança,https://www.noh.com.br/ajuda-categorias/sobre-...


In [4]:
df["conteudo"] = df["conteudo"].apply(lambda x: " ".join(x.split()))

In [5]:
print(df.loc[0, "conteudo"])

A Noh é uma conta para o casal que junto, pode tudo , e busca um lugar pra controlar e juntar suas finanças. Sabe aquela assinatura da Netflix, pedido no iFood, compras do mercado ou qualquer outra conta que sempre dá dor de cabeça na hora de pagar juntos? Estamos aqui para acabar com isso, trazendo mais transparência, justiça e simplicidade para as finanças do casal. Com a Noh vocês podem fazer pagamentos via Pix, boleto ou cartão pré-pago, além de terem acesso ao extrato para saber quanto cada um contribuiu. Assim, vocês percebem que, juntos, podem tudo . Baixe o nosso app na Apple Store ou Google Play para dizer adeus às planilhas, bloco de notas e discussões na hora de pagar as contas.


Vou gerar os embeddings usando uma concatenação de título do artigo e conteúdo 

In [6]:
df["texto_completo"] = df["titulo"] + " " + df["conteudo"]

In [7]:
df.head(2)

,url,titulo,conteudo,artigos_relacionados,categoria,subtitulo_categoria,link_categoria,texto_completo
0,https://www.noh.com.br/ajuda/o-que-e-a-noh,O que é a Noh?,"A Noh é uma conta para o casal que junto, pode...",[https://www.noh.com.br/ajuda/como-funciona-o-...,Sobre Nohs,Saiba mais sobre a Noh e a nossa segurança,https://www.noh.com.br/ajuda-categorias/sobre-...,O que é a Noh? A Noh é uma conta para o casal ...
1,https://www.noh.com.br/ajuda/os-meus-dados-e-o...,Os meus dados e o meu dinheiro estão seguros?,"Sim, os seus dados e dinheiro estão seguros co...",[https://www.noh.com.br/ajuda/como-funciona-o-...,Sobre Nohs,Saiba mais sobre a Noh e a nossa segurança,https://www.noh.com.br/ajuda-categorias/sobre-...,Os meus dados e o meu dinheiro estão seguros? ...


In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

c:\Users\daniel.senna\AppData\Local\anaconda3\envs\noh-faq\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Index 1 - modelo de embeddings small

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings_small = OpenAIEmbeddings(model="text-embedding-3-small")

In [11]:
import time

index_name = "noh-faq-index"  

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

else:
    print(f"'{index_name}' já existe.")

index_small = pc.Index(index_name)

'noh-faq-index' já existe.


In [12]:
from langchain_pinecone import PineconeVectorStore

vector_store_small = PineconeVectorStore(index=index_small, embedding=embeddings_small)

In [13]:
from uuid import uuid4

from langchain_core.documents import Document

In [15]:
documentos_small = []
ids_small = []

In [16]:
for i, row in df.iterrows():
    documento = Document(
        page_content=row["texto_completo"],
        metadata={
            "titulo_artigo": row["titulo"],
            "url_artigo": row["url"],
            "conteudo_artigo": row["conteudo"],
            "titulo_categoria": row["categoria"],
            "subtitulo_categoria": row["subtitulo_categoria"],
            "link_categoria": row["link_categoria"],
            "artigos_relacionados": row["artigos_relacionados"],
        }
    )
    documentos_small.append(documento)
    ids_small.append(str(uuid4()))  # Gerando uuid para servir de id do vetor

In [17]:
#checando um documento
documentos_small[0]

Document(metadata={'titulo_artigo': 'O que é a Noh?', 'url_artigo': 'https://www.noh.com.br/ajuda/o-que-e-a-noh', 'conteudo_artigo': 'A Noh é uma conta para o casal que junto, pode tudo , e busca um lugar pra controlar e juntar suas finanças. Sabe aquela assinatura da Netflix, pedido no iFood, compras do mercado ou qualquer outra conta que sempre dá dor de cabeça na hora de pagar juntos? Estamos aqui para acabar com isso, trazendo mais transparência, justiça e simplicidade para as finanças do casal. Com a Noh vocês podem fazer pagamentos via Pix, boleto ou cartão pré-pago, além de terem acesso ao extrato para saber quanto cada um contribuiu. Assim, vocês percebem que, juntos, podem tudo . Baixe o nosso app na Apple Store ou Google Play para dizer adeus às planilhas, bloco de notas e discussões na hora de pagar as contas.', 'titulo_categoria': 'Sobre Nohs', 'subtitulo_categoria': 'Saiba mais sobre a Noh e a nossa segurança', 'link_categoria': 'https://www.noh.com.br/ajuda-categorias/sob

In [18]:
#checando um id 
ids_small[0]

'033fca6f-4ce9-4e5f-8554-a6ac78fe6c53'

In [19]:
#vou deixar um json salvo já no formatinho dos dados que serão usados para criar os embeddings
dados_em_json_small = []

for documento, id in zip(documentos_small, ids_small):
    dados_em_json_small.append({
        "id": id,
        "conteudo_origem": documento.page_content,
        "metadados": documento.metadata
    })

with open("noh_faq_embeddings_small.json", "w", encoding="utf-8") as f:
    json.dump(dados_em_json_small, f, ensure_ascii=False, indent=4)

In [20]:
#gerando os embeddings e subindo para o pinecone com langchain
vector_store_small.add_documents(documents=documentos_small, ids=ids_small)

['033fca6f-4ce9-4e5f-8554-a6ac78fe6c53',
 '7d81a48a-f992-4ac0-af9f-2fbe8e97d93a',
 '27507e6b-f5f5-4bae-a9b0-14350fe733bf',
 '4370866e-d739-4d6c-8174-89cca7a47cf8',
 'c3315b76-9d72-4e91-b607-f4bb67b75f87',
 'a2fa6ca1-93fb-491b-9db8-28c1063a79f8',
 '630292a1-8968-4d19-ad97-50adce171631',
 '7859417d-4643-47be-881f-a505d5a039bd',
 '8dcf7929-d4ce-44c2-9cf5-27427eeafbea',
 '11dce9b8-6031-4900-a971-cf75d244ee69',
 '8eaa44a5-7dec-4ab4-8b1d-316eb16ada34',
 'ab72ea3f-dbf9-44d4-bd0a-2fabdb401e57',
 '3f1b297d-8dcc-4d27-8496-a1da10d82e4a',
 '36cc101e-d421-41ff-b406-7a813dc1cb8b',
 '92ec0b33-6eba-4ac5-afda-3763c741e31b',
 '501caff7-2150-4de8-841b-48acfa5c2a6e',
 'c044d54c-fd0f-4441-86d8-98ff44775eb5',
 'b478ac4c-681e-4f21-8141-0a0e0fb586f2',
 '77c63d11-cd46-4eff-bce7-6de116e8d915',
 '1e460016-b197-4c93-b8c0-39f6a7144f1d',
 '8255e061-05d5-4adf-8180-5be705cf51a6',
 'e2edf81a-7abf-47e2-806c-d20aac744790',
 '4b807b15-9bea-40df-acf5-9b59972f133a',
 '2f8a5cc5-f165-4391-8037-75262dd81b59',
 '5a484c6a-333a-

In [21]:
#checando se o index no pinecone foi criado
pc.describe_index("noh-faq-index")

{
    "name": "noh-faq-index",
    "dimension": 1536,
    "metric": "cosine",
    "host": "noh-faq-index-0c15b22.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [23]:
#checando um item do index
vector_data_small = index_small.fetch(["033fca6f-4ce9-4e5f-8554-a6ac78fe6c53"])
print(vector_data_small)

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'033fca6f-4ce9-4e5f-8554-a6ac78fe6c53': {'id': '033fca6f-4ce9-4e5f-8554-a6ac78fe6c53',
                                                      'metadata': {'artigos_relacionados': ['https://www.noh.com.br/ajuda/como-funciona-o-programa-de-indicacao-da-noh',
                                                                                            'https://www.noh.com.br/ajuda/como-e-declarado-o-imposto-de-renda-da-conta-conjunta',
                                                                                            'https://www.noh.com.br/ajuda/onde-encontro-meu-informe-de-rendimentos',
                                                                                            'https://www.noh.com.br/ajuda/quem-pode-criar-uma-conta-na-noh',
                                                                                            'https://www.noh.com.br/ajuda/os-meus-dados-e-o-meu-dinheiro-estao-seguros'],
              

In [60]:
#testando uma query
start_time_small = time.time()
results_small = vector_store_small.similarity_search_with_score(
    "preciso pagar pra ter conta na Noh?",
    k=5
    )
end_time_small = time.time()
tempo_small = end_time_small - start_time_small

In [62]:
print(f"Tempo modelo small: {tempo_small:.4f} segundos")

Tempo modelo small: 1.7872 segundos


In [61]:
for i  in results_small:
    print(i[0].page_content)
    print(i[1])

O que acontece se eu não quiser assinar a Noh? Ao criar a conta, damos um prazo para testarem o app e decidirem se ele faz sentido nas finanças compartilhadas de vocês, mas só assinantes poderão usar a Noh. Caso não queiram assinar, vocês poderão desativar suas contas pelo próprio app. Ao desativá-las, não será mais possível fazer pagamentos com a Noh, os cartões (virtual e físico) serão cancelados e vocês não terão mais acesso ao app.
0.708716094
Como faço pra criar conta? Para você criar uma conta na Noh e começar a dividir os gastos com seu amor, siga estes passos: Baixe o app da Noh na App Store ou Google Play ; Insira seu telefone com DDD; Digite o código que irá receber por SMS; Complete o cadastro com seus dados pessoais e endereço; Envie o seu documento, RG ou CNH, e selfie para validação da conta; Adicione o seu amor e crie seu Noh. Pronto! Agora é só esperar, assim que terminarmos a análise para abertura da sua conta, vocês receberão uma notificação do app.
0.699805677
Quem p

### Index 2 - modelo de embeddings large

In [27]:
from langchain_openai import OpenAIEmbeddings

embeddings_large = OpenAIEmbeddings(model="text-embedding-3-large")

In [28]:
import time

index_name = "noh-faq-index-3072"  

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

else:
    print(f"'{index_name}' já existe.")

index_large = pc.Index(index_name)

'noh-faq-index-3072' já existe.


In [30]:
from langchain_pinecone import PineconeVectorStore

vector_store_large = PineconeVectorStore(index=index_large, embedding=embeddings_large)

In [29]:
documentos_large = []
ids_large = []

In [31]:
for i, row in df.iterrows():
    documento = Document(
        page_content=row["texto_completo"],
        metadata={
            "titulo_artigo": row["titulo"],
            "url_artigo": row["url"],
            "conteudo_artigo": row["conteudo"],
            "titulo_categoria": row["categoria"],
            "subtitulo_categoria": row["subtitulo_categoria"],
            "link_categoria": row["link_categoria"],
            "artigos_relacionados": row["artigos_relacionados"],
        }
    )
    documentos_large.append(documento)
    ids_large.append(str(uuid4()))  # Gerando uuid para servir de id do vetor

In [32]:
#vou deixar um json salvo já no formatinho dos dados que serão usados para criar os embeddings
dados_em_json = []

for documento, id in zip(documentos_large, ids_large):
    dados_em_json.append({
        "id": id,
        "conteudo_origem": documento.page_content,
        "metadados": documento.metadata
    })

with open("noh_faq_embeddings_large.json", "w", encoding="utf-8") as f:
    json.dump(dados_em_json, f, ensure_ascii=False, indent=4)

In [33]:
#gerando os embeddings e subindo para o pinecone com langchain
vector_store_large.add_documents(documents=documentos_large, ids=ids_large)

['b8c71050-d3d0-44f9-b589-c10d06279381',
 'b1b01ca7-5330-46c7-892b-bc03a9456c35',
 '7a3a1a24-636a-47cc-9af0-76461107b499',
 'fd27bdfb-c719-4f6d-87d7-152aacb31b2d',
 '7b09aa34-72a5-4d51-8122-5ef40d62a86f',
 '2e0581f5-e290-4149-81a4-d27fb4fe1b7a',
 'd81bc5cd-457f-4323-bde2-19bce514be56',
 '3bf61dbc-2808-4a46-8df5-f16cce4e2c95',
 '82048b3f-bba4-4bc3-b949-f7522fef5252',
 '3765197d-61d5-4ac6-a8df-d0195aa2c80c',
 'e577f5cc-3d00-48d2-89d1-9d2fcd9196a8',
 'ab0e8dfd-b03e-4759-a090-f4354765817a',
 '1dc9e87d-8e94-4efc-97a6-5f36f99101a7',
 '8fad822b-02b4-4858-9b4c-29790f9e0f41',
 '0dd00504-3e45-41d3-9d07-c67e059366a3',
 '9daf5275-9a71-4679-9c26-618b6882da6f',
 '6179528f-b1d1-4590-b762-209519b4bbee',
 'b902e11b-9638-4058-97c9-2c42b9fc3348',
 'b1d21941-7809-41c0-953a-32902d613239',
 'ca9d9aac-91d3-4104-8b57-f673946e000c',
 '836ac7a9-422b-4870-b4bb-869032b4daa8',
 '3be58b0e-1d0e-47c4-bc22-7a38b9497df7',
 '9cd83363-3096-4216-8499-b917eeb682e0',
 '66b0179d-5653-489a-b3cc-027c42fc3d13',
 '9a38d90f-6703-

In [34]:
#checando se o index no pinecone foi criado
pc.describe_index("noh-faq-index-3072")

{
    "name": "noh-faq-index-3072",
    "dimension": 3072,
    "metric": "cosine",
    "host": "noh-faq-index-3072-0c15b22.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [35]:
#checando um item do index
vector_data_large = index_large.fetch(["b8c71050-d3d0-44f9-b589-c10d06279381"])
print(vector_data_large)

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'b8c71050-d3d0-44f9-b589-c10d06279381': {'id': 'b8c71050-d3d0-44f9-b589-c10d06279381',
                                                      'metadata': {'artigos_relacionados': ['https://www.noh.com.br/ajuda/como-funciona-o-programa-de-indicacao-da-noh',
                                                                                            'https://www.noh.com.br/ajuda/como-e-declarado-o-imposto-de-renda-da-conta-conjunta',
                                                                                            'https://www.noh.com.br/ajuda/onde-encontro-meu-informe-de-rendimentos',
                                                                                            'https://www.noh.com.br/ajuda/quem-pode-criar-uma-conta-na-noh',
                                                                                            'https://www.noh.com.br/ajuda/os-meus-dados-e-o-meu-dinheiro-estao-seguros'],
              

In [57]:
#testando uma query
start_time_large = time.time()
results_large = vector_store_large.similarity_search_with_score(
    "preciso pagar pra ter conta na Noh?",
    k=5
    )
end_time_large = time.time()
tempo_large = end_time_large - start_time_large

In [59]:
print(f"Tempo modelo large: {tempo_large:.4f} segundos")

Tempo modelo large: 2.2862 segundos


In [58]:
for i  in results_large:
    print(i[0].page_content)
    print(i[1])

Quanto custa a Noh? A Noh tem uma mensalidade de R$14,00 por casal (R$ 7,00 por pessoa). Esse valor é o que vai nos ajudar a continuar crescendo de forma sustentável, sem a necessidade de vender seus dados, ou exibir anúncios de outros produtos que poluam sua experiência no aplicativo. Queremos que nossa fonte de receita venha do aplicativo e cartão que criamos, que é o símbolo do nosso movimento para acabar com as gambiarras dos casais que tentam adaptar ferramentas individuais para uma vida conjunta.
0.663752198
O que acontece se eu não quiser assinar a Noh? Ao criar a conta, damos um prazo para testarem o app e decidirem se ele faz sentido nas finanças compartilhadas de vocês, mas só assinantes poderão usar a Noh. Caso não queiram assinar, vocês poderão desativar suas contas pelo próprio app. Ao desativá-las, não será mais possível fazer pagamentos com a Noh, os cartões (virtual e físico) serão cancelados e vocês não terão mais acesso ao app.
0.657431245
Por que a mensalidade? A Noh